In [0]:
spark.sql("DROP TABLE IF EXISTS carros")

DataFrame[]

In [0]:
# overwrite com inferschema
carros = spark.read.load("/Volumes/workspace/default/data_files/carros.csv",
                     format="csv", sep=";", inferSchema=True, header=True)
carros.write.mode("overwrite").saveAsTable("carros")                     

In [0]:
%%sql
-- carros já existe
select * from carros

Consumo,Cilindros,Cilindradas,RelEixoTraseiro,Peso,Tempo,TipoMotor,Transmissao,Marchas,Carburadors,HP
21,6,160,39,262,1646,0,1,4,4,110
21,6,160,39,2875,1702,0,1,4,4,110
228,4,108,385,232,1861,1,1,4,1,93
214,6,258,308,3215,1944,1,0,3,1,110
187,8,360,315,344,1702,0,0,3,2,175
181,6,225,276,346,2022,1,0,3,1,105
143,8,360,321,357,1584,0,0,3,4,245
244,4,1467,369,319,20,1,0,4,2,62
228,4,1408,392,315,229,1,0,4,2,95
192,6,1676,392,344,183,1,0,4,4,123


DataFrame[Consumo: int, Cilindros: int, Cilindradas: int, RelEixoTraseiro: int, Peso: int, Tempo: int, TipoMotor: int, Transmissao: int, Marchas: int, Carburadors: int, HP: int]

In [0]:
from pyspark.sql.functions import col, when
carros_df = spark.table("carros")
# nome da pasta
carros_df.write.format("delta").save("/Volumes/workspace/default/data_files/carros_delta")

In [0]:
delta_df = spark.read.format("delta").load("/Volumes/workspace/default/data_files/carros_delta")
delta_df.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [0]:
#atualiza tipo de motor
delta_df = delta_df.withColumn('TipoMotor',when(col('TipoMotor') == 0, 2).otherwise(col('TipoMotor')))

In [0]:
#grava, carrega e exibe
delta_df.write.format("delta").mode("overwrite").save("/Volumes/workspace/default/data_files/carros_delta")
updated_delta_df = spark.read.format("delta").load("/Volumes/workspace/default/data_files/carros_delta")
updated_delta_df.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        2|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        2|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        2|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [0]:
%%sql
SELECT version, timestamp, operation, operationMetrics
FROM (
  DESCRIBE HISTORY delta.`/Volumes/workspace/default/data_files/carros_delta`
)


version,timestamp,operation,operationMetrics
1,2025-07-27T22:58:50.000Z,WRITE,"Map(numFiles -> 1, numRemovedFiles -> 1, numRemovedBytes -> 3571, numOutputRows -> 32, numOutputBytes -> 3572)"
0,2025-07-27T22:58:47.000Z,WRITE,"Map(numFiles -> 1, numOutputRows -> 32, numOutputBytes -> 3571)"


DataFrame[version: bigint, timestamp: timestamp, operation: string, operationMetrics: map<string,string>]

In [0]:
# "rollback" da tabela
version0_df = spark.read.format("delta").option("versionAsOf", 0).load("/Volumes/workspace/default/data_files/carros_delta")
version0_df.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [0]:
#versao atual continua
updated_delta_df.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        2|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        2|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        2|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [0]:
version_default = spark.read.format("delta").load("/Volumes/workspace/default/data_files/carros_delta")
version_default.show()

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        2|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        2|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        2|          0|      3|          2|175|
|    181|        6|        225|            276| 346| 2022|        1|          0|      3|          1|105|
|    143|        8|        360|            321| 357| 15

In [0]:
#apaga delta
dbutils.fs.rm("/Volumes/workspace/default/data_files/carros_delta", recurse=True)

True